In [1]:
import pandas as pd

In [2]:
# name of each discipline
field_name = pd.read_csv("../data/mag/FieldsOfStudy.txt", sep="\t",
                        names = ["FieldOfStudyId", "Rank", "NormalizedName", "DisplayName",
                                 "MainType", "Level", "PaperCount", "CitationCount", "CreatedDate"],
                       usecols=['FieldOfStudyId','DisplayName']).rename(columns={'DisplayName':'Field'})

In [3]:
%%time
# calculated discipline of all scientists in MAG
field_parent = pd.read_csv("/scratch/fl1092/capstone/conflated/AuthorFields.csv",sep='\t',
        usecols=['NewAuthorId', 'Parent'], dtype={'NewAuthorId':int, 'Parent':int}).rename(
    columns={'Parent':'FieldOfStudyId'})
assert(field_parent.FieldOfStudyId.nunique() == 19)

CPU times: user 29.3 s, sys: 6.83 s, total: 36.1 s
Wall time: 36.3 s


In [4]:
def getJournalField(df):
    jfield = (
        pd.concat([pd.read_csv('/scratch/fl1092/capstone/bigmem/Elsevier_journal_top_fields.csv',sep='\t',
                     usecols=['issn','Parent','Score'],dtype={'issn':str,'Parent':int,'Score':float})],
                  ignore_index=True, sort=False)
        .assign(issn = lambda _: _.issn.apply(lambda x: '0'*(8-len(x)) + x))
        .sort_values(by='Score',ascending=False).drop_duplicates(subset=['issn'],keep='first')
        .merge(field_name, left_on='Parent', right_on='FieldOfStudyId')
    )
    
    jfield = jfield[['issn','FieldOfStudyId','Field']]
    
    df = df.merge(jfield, on='issn')
    
    return df
    
def getScientistField(df):
    df = df.merge(field_parent, on='NewAuthorId')
    df = df.merge(field_name, on='FieldOfStudyId')
    
    return df

In [5]:
def printTable(table, multiplier=1):
    # multiplier: when printing table, print percentage value
    
    top15_fields = pd.DataFrame({'Discipline':['Biology', 'Business', 'Chemistry', 'Computer science', 'Economics',
                                     'Engineering', 'Geology', 'Materials science', 'Mathematics', 'Medicine',
                                     'Philosophy', 'Physics', 'Political science' ,'Psychology', 'Sociology']})
    table = table.merge(
        top15_fields.rename(columns={'Discipline':'Field'}),
        on='Field')
    
    t1 = (
        table.Field.value_counts().reset_index()
        .rename(columns={'index':'Discipline', 'Field':'Number of editors'})
    )
    t2 = (
        table.Field.value_counts(normalize=True).reset_index()
        .rename(columns={'index':'Discipline', 'Field':'Percentage'})
    )
    
    t1 = top15_fields.merge(t1, on=['Discipline']).merge(t2, on=['Discipline']).fillna(0)
    
    t1 = (
        t1.assign(Percentage = t1.Percentage.apply(lambda x: x*multiplier))
        .sort_values('Percentage', ascending=False)
    )
    
    print(f'Total: {t1["Number of editors"].sum()}')
    
    return t1

### MAG editors

In [6]:
editors = pd.read_csv("/scratch/fl1092/capstone/elsevier/editors.csv", sep='\t',
                     usecols=["NewAuthorId", "issn", "start_year", "end_year"],
                     dtype={"NewAuthorId":int, "issn":str, "start_year":int, "end_year":int})

In [7]:
%%time
table1 = getScientistField(editors)

CPU times: user 39.7 s, sys: 10.5 s, total: 50.2 s
Wall time: 50.3 s


### Gender editors

In [8]:
editors = pd.read_csv("/scratch/fl1092/capstone/temp/EditorsUnion.csv",sep='\t',
                      usecols=['ElEditorID','issn','start_year','end_year','gender'],
                     dtype={'ElEditorID':int,'issn':str,'start_year':int,'end_year':int,'gender':str})

In [9]:
%%time
table2 = getJournalField(editors)

CPU times: user 110 ms, sys: 40.8 ms, total: 151 ms
Wall time: 151 ms


### All editors

In [10]:
def getFirstLastName(df, col):
    df = df.assign(First = df[col].apply(lambda x: x.split()[0]))
    df = df.assign(last = df[col].apply(lambda x: x.split()[-1]))
    
    return df

def addDataSet(df, editors):
    df = df.merge(editors.assign(Exist=1), on=['issn', 'First', 'last'], how='left').fillna(0)
    
    df = df[df.Exist == 0]
    
    return df

editors = pd.read_csv('/scratch/fl1092/capstone/revise/EditorsRawData.csv',sep='\t',
                     usecols=['issn','last','First'], dtype={'issn':str})

toAdd1 = (
    pd.read_csv('/scratch/fl1092/capstone/temp/ManualSociologyEditors.csv',sep='\t',
                dtype={'issn':str,'start_year':int,'end_year':int,'gender':str,'Name':str})
    .pipe(getFirstLastName, 'Name')
    .pipe(addDataSet, editors)
)

editors = (
    pd.concat([editors, toAdd1[['issn','last','First']] ])
)

toAdd2 = (
    pd.read_csv("/scratch/fl1092/capstone/temp/EditorsUnion.csv",sep='\t',
                usecols=['issn','Name'],
                dtype={'issn':str,'Name':str})
    .drop_duplicates()
    .pipe(getJournalField)
    .query('FieldOfStudyId == 144133560')
    .pipe(getFirstLastName, 'Name')
    .pipe(addDataSet, editors)
    .drop('Name', axis=1)
    .drop_duplicates()
)

editors = (
    pd.concat([editors, toAdd2[['issn','last','First']] ])
)
assert(editors.duplicated().any()==False)

# (105962, 3) 1816
# (131, 7)
# (99, 8)
# (106061, 3)

In [11]:
%%time
table3 = getJournalField(editors)

CPU times: user 111 ms, sys: 36.9 ms, total: 148 ms
Wall time: 147 ms


# Tables

In [12]:
t1 = printTable(table1)
t2 = printTable(table2)
t3 = printTable(table3)

Total: 19698
Total: 80776
Total: 102964


In [13]:
t1.to_csv('../data/supplementary/EditorsMAGTable.csv',index=False,sep='\t')
t2.to_csv('../data/supplementary/EditorsGenderTable.csv',index=False,sep='\t')
t3.to_csv('../data/supplementary/EditorsRawTable.csv',index=False,sep='\t')

In [14]:
t1

,Discipline,Number of editors,Percentage
0,Biology,3949,0.200477
4,Economics,3537,0.179561
3,Computer science,2318,0.117677
11,Physics,1725,0.087572
8,Mathematics,1633,0.082902
9,Medicine,1357,0.068890
2,Chemistry,1216,0.061732
13,Psychology,1039,0.052746
12,Political science,786,0.039903
5,Engineering,761,0.038633


In [15]:
t2

,Discipline,Number of editors,Percentage
0,Biology,22474,0.278226
9,Medicine,17442,0.215930
4,Economics,7129,0.088256
3,Computer science,6190,0.076632
11,Physics,5783,0.071593
2,Chemistry,4409,0.054583
8,Mathematics,4040,0.050015
13,Psychology,3831,0.047427
5,Engineering,3024,0.037437
12,Political science,2494,0.030876


In [16]:
t3

,Discipline,Number of editors,Percentage
0,Biology,27914,0.271104
9,Medicine,18005,0.174867
11,Physics,9348,0.090789
4,Economics,8649,0.084000
3,Computer science,8226,0.079892
2,Chemistry,7238,0.070296
8,Mathematics,5977,0.058049
13,Psychology,4248,0.041257
5,Engineering,3839,0.037285
12,Political science,2875,0.027922


# Anonymize editor annual self-publication count and rate

In [1]:
import pandas as pd

In [2]:
%%time
authorProd = (
    pd.read_csv("/scratch/fl1092/capstone/conflated/PaperCountAnnual.csv", sep='\t')
    .rename(columns={'PaperCount':'Productivity'})
)

(325866578, 3)
CPU times: user 42.3 s, sys: 13.7 s, total: 56 s
Wall time: 56.4 s


In [3]:
%%time
author_career = pd.read_csv('/scratch/fl1092/capstone/conflated/AuthorEraDisp.csv',
            sep='\t', memory_map=True,
            usecols=['NewAuthorId', 'Yfp', 'Ylp', 'Parent'],
            dtype={'NewAuthorId':int, 'Yfp':int, 'Ylp':int, 'Parent':int})
print(author_career.shape) # 120948543

(120948543, 4)
CPU times: user 24.1 s, sys: 5.1 s, total: 29.2 s
Wall time: 29.4 s


In [4]:
editor_gender = pd.read_csv('/scratch/fl1092/capstone/revise/EditorGender.csv',sep='\t',
                           dtype={'NewAuthorId':int,'issn':str, 'start_year':int, 'end_year':int, 'gender':str})
editor_gender = editor_gender[editor_gender.start_year < 2015]

In [5]:
%%time
editor_attribute = editor_gender.merge(author_career, on='NewAuthorId')

CPU times: user 38.6 s, sys: 10.9 s, total: 49.5 s
Wall time: 49.7 s


In [6]:
def getFull(df, left, right):
    full = []
    df = df.assign(Year0 = lambda df: df.start_year-1)
    
    for year in range(df[left].min(), df[right].max() + 1):
        (
            full.append(df[(df[left] <= year) & (df[right] >= year)]
                        .assign(EditorYear = lambda df: year-df.Year0)
                       )
        )
    
    return pd.concat(full, ignore_index=True, sort=False)

In [7]:
editors = pd.read_csv(
    '/scratch/fl1092/capstone/revise/EditorsLeftRight.csv',sep='\t',
    usecols=['NewAuthorId','issn','start_year','left','right']
).assign(Year0 = lambda df: df.start_year - 1)

In [8]:
full_range = getFull(editors, 'left', 'right').drop(
    ['start_year','left','right'], axis=1)
full_range.shape, full_range.EditorYear.min(), full_range.EditorYear.max()

((119553, 4), -4, 5)

In [9]:
papers = (
    pd.read_csv(
        f"/scratch/fl1092/capstone/revise/EditorJournalPublicationNoEditorial/5.csv",sep='\t',
        dtype={'NewAuthorId':int,'issn':str,'Year0':int,
               'PaperId':int,'Year':int,'Editorial':int}
    )
    .assign(EditorYear = lambda df: df.Year - df.Year0)
)

assert((papers.Editorial == 0).all())
assert(papers.EditorYear.min() == -4)
assert(papers.EditorYear.max() == 5)

paper_count = (
    papers.groupby(['NewAuthorId','issn','EditorYear']).PaperId.nunique()
    .reset_index()
    .rename(columns={'PaperId':'JournalCount'})
)

paper_count = (
    full_range.merge(paper_count, on=['NewAuthorId','issn','EditorYear'], how='left')
    .assign(Year = lambda df: df.Year0 + df.EditorYear)
    .merge(authorProd, on=['NewAuthorId','Year'], how='left')
    .fillna({'JournalCount':0})
    
    .assign(Percentage=lambda df: df.JournalCount/df.Productivity)
    .fillna({'Percentage':0})
)

(41354, 7)
(24470, 4)
(119553, 8)


In [39]:
toPlot = paper_count.merge(editor_attribute, on=['NewAuthorId','issn'])

# anonymize issn and authorId #
anoIssn = toPlot[['issn']].drop_duplicates().reset_index().rename(columns={'index':'AnoIssn'})
anoID = toPlot[['NewAuthorId']].drop_duplicates().reset_index().rename(columns={'index':'AnoID'})

toPlot = (
    toPlot.merge(anoIssn, on='issn') 
    .merge(anoID, on='NewAuthorId')
    .drop(['issn','NewAuthorId'], axis=1)
    .rename(columns={'AnoIssn':'issn', 'AnoID':'NewAuthorId'})
)
########
toPlot.shape

(119553, 14)

In [42]:
toPlot.reset_index().to_csv('../data/figure_3_selfpub_regression.csv',
                                 columns=['NewAuthorId','issn','EditorYear','JournalCount','Productivity','Percentage',
                                         'gender','start_year'], index=False)